## Analisis de datos

#### Creamos un dataframe con los datos de general data y lo fragmentamos y guardamos en un csv por departamento

Obtenemos los datos del csv

In [10]:
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Carga datos en un DataFrame
general_data = pd.read_csv('Reporte_Capturas_Polic_a_Nacional_20241025.csv', low_memory=False)
# Eliminar columnas espurias
general_data =  general_data[~general_data["DEPARTAMENTO"].isin(["QUITO", "MADRID","NEW YORK","NEW JERSEY","FLORIDA"])]
# Convierte la columna FECHA_HECHO a formato datetime
general_data['FECHA_HECHO'] = pd.to_datetime(general_data['FECHA_HECHO'], format="%d/%m/%Y")
general_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 839775 entries, 0 to 839792
Data columns (total 8 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   DEPARTAMENTO                  839775 non-null  object        
 1   MUNICIPIO                     839775 non-null  object        
 2   CODIGO_DANE                   839775 non-null  object        
 3   FECHA_HECHO                   839775 non-null  datetime64[ns]
 4   GENERO                        839775 non-null  object        
 5   DESCRIPCION_CONDUCTA_CAPTURA  839775 non-null  object        
 6   GRUPO_ETARIO                  839775 non-null  object        
 7   CANTIDAD                      839775 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 57.7+ MB


In [18]:
def csv_by_filter(data, filter_column, output_folder, out = True):
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(output_folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter_column, data in general_data.groupby(filter_column):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(output_folder, f"{filter_column}.csv")
        if (out):
            print(f"Guardando {output_file}... con {len(data)} filas")
        # Guarda el DataFrame filtrado como CSV
        data.to_csv(output_file, index=False)
    if (out):
        print("Archivos CSV generados en la carpeta:", output_folder)

# Filtra y guarda un CSV para cada departamento
csv_by_filter(general_data, "DEPARTAMENTO", 'departamentos_base', out = True)

Guardando departamentos_base\AMAZONAS.csv... con 3042 filas
Guardando departamentos_base\ANTIOQUIA.csv... con 115316 filas
Guardando departamentos_base\ARAUCA.csv... con 4261 filas
Guardando departamentos_base\ATLÁNTICO.csv... con 35313 filas
Guardando departamentos_base\BOLÍVAR.csv... con 34910 filas
Guardando departamentos_base\BOYACÁ.csv... con 27861 filas
Guardando departamentos_base\CALDAS.csv... con 29073 filas
Guardando departamentos_base\CAQUETÁ.csv... con 10663 filas
Guardando departamentos_base\CASANARE.csv... con 9640 filas
Guardando departamentos_base\CAUCA.csv... con 20747 filas
Guardando departamentos_base\CESAR.csv... con 17146 filas
Guardando departamentos_base\CHOCÓ.csv... con 4969 filas
Guardando departamentos_base\CUNDINAMARCA.csv... con 115954 filas
Guardando departamentos_base\CÓRDOBA.csv... con 28266 filas
Guardando departamentos_base\GUAINÍA.csv... con 1115 filas
Guardando departamentos_base\GUAJIRA.csv... con 8758 filas
Guardando departamentos_base\GUAVIARE.csv.

In [19]:
import os
import pandas as pd

def csv_by_filter(data, filter_column, output_folder, out=True):
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(output_folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter_value, filtered_data in data.groupby(filter_column):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(output_folder, f"{filter_value}.csv")
        if out:
            print(f"Guardando {output_file}... con {len(filtered_data)} filas")
        
        # Convierte las columnas de tipo object a tipo category y luego a dummies
        categorical_columns = filtered_data.select_dtypes(include=['object']).columns
        filtered_data = pd.get_dummies(filtered_data, columns=categorical_columns, prefix=categorical_columns, drop_first=True)

        # Guarda el DataFrame filtrado como CSV
        filtered_data.to_csv(output_file, index=False)
    
    if out:
        print("Archivos CSV generados en la carpeta:", output_folder)

# Filtra y guarda un CSV para cada departamento
csv_by_filter(general_data, "DEPARTAMENTO", 'departamentos_codificados', out=True)


Guardando departamentos_codificados\AMAZONAS.csv... con 3042 filas
Guardando departamentos_codificados\ANTIOQUIA.csv... con 115316 filas
Guardando departamentos_codificados\ARAUCA.csv... con 4261 filas
Guardando departamentos_codificados\ATLÁNTICO.csv... con 35313 filas
Guardando departamentos_codificados\BOLÍVAR.csv... con 34910 filas
Guardando departamentos_codificados\BOYACÁ.csv... con 27861 filas
Guardando departamentos_codificados\CALDAS.csv... con 29073 filas
Guardando departamentos_codificados\CAQUETÁ.csv... con 10663 filas
Guardando departamentos_codificados\CASANARE.csv... con 9640 filas
Guardando departamentos_codificados\CAUCA.csv... con 20747 filas
Guardando departamentos_codificados\CESAR.csv... con 17146 filas
Guardando departamentos_codificados\CHOCÓ.csv... con 4969 filas
Guardando departamentos_codificados\CUNDINAMARCA.csv... con 115954 filas
Guardando departamentos_codificados\CÓRDOBA.csv... con 28266 filas
Guardando departamentos_codificados\GUAINÍA.csv... con 1115 fi

In [15]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Carga los datos
cauca = pd.read_csv('departamentos/CAUCA.csv', low_memory=False)

# Asegúrate de que las columnas sean de tipo string
cauca[["DEPARTAMENTO", "MUNICIPIO", "CODIGO_DANE", "GENERO", "DESCRIPCION_CONDUCTA_CAPTURA", "GRUPO_ETARIO"]] = cauca[["DEPARTAMENTO", "MUNICIPIO", "CODIGO_DANE", "GENERO", "DESCRIPCION_CONDUCTA_CAPTURA", "GRUPO_ETARIO"]].astype(str)

# Codificación One-Hot
codificador = OneHotEncoder(sparse_output=False)
codificacion = codificador.fit_transform(cauca[["DEPARTAMENTO", "MUNICIPIO", "CODIGO_DANE", "GENERO", "DESCRIPCION_CONDUCTA_CAPTURA", "GRUPO_ETARIO"]])

# Crear nombres de columnas para el DataFrame resultante
nombres_columnas = []
for i, cat in enumerate(codificador.categories_):
    print("x")
    nombres_columnas.extend([f"{codificador.feature_names_in_[i]}_{val}" for val in cat])

# Crear el DataFrame con los resultados de la codificación
nueva_data = pd.DataFrame(codificacion, columns=nombres_columnas)

# Unir los dos DataFrames
cauca_completo = pd.concat([cauca.reset_index(drop=True), nueva_data.reset_index(drop=True)], axis=1)

# Guardar el DataFrame combinado en un archivo CSV
output_file = 'cauca_completo.csv'
cauca_completo.to_csv(output_file, index=False)
cauca_completo.info()

x
x
x
x
x
x
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20747 entries, 0 to 20746
Columns: 266 entries, DEPARTAMENTO to GRUPO_ETARIO_MENORES
dtypes: float64(258), int64(1), object(7)
memory usage: 42.1+ MB
